# Translation Chain (v1 – June 2025 API)

A reproducible pipeline that…

1. **Pre-processes** an input text  
2. **Generates & refines** a translation in `n` feedback loops  
3. **Post-processes** the best translation against canonical references  
4. Lets you **apply edits**

In [1]:
!pip install -U "openai>=1.10.0"

import os, json, textwrap
from openai import OpenAI
from google.colab import userdata

openai_api_key = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

MODEL_MAIN   = "gpt-4.1"
MODEL_SEARCH = "gpt-4.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.0/755.0 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.91.0
    Uninstalling openai-1.91.0:
      Successfully uninstalled openai-1.91.0


# Input text
Paste or edit your source text below.


In [2]:
SOURCE_TEXT = """
Alle Sätze sind Resultate von Wahrheitsoperationen mit den Elementarsätzen.
Die Wahrheitsoperation ist die Art und Weise, wie aus den Elementarsätzen die Wahrheitsfunktion entsteht.
Nach dem Wesen der Wahrheitsoperation wird auf die gleiche Weise, wie aus den Elementarsätzen ihre Wahrheitsfunktion, aus Wahrheitsfunktionen eine Neue. Jede Wahrheitsoperation erzeugt aus Wahrheitsfunktionen von Elementarsätzen wieder eine Wahrheitsfunktion von Elementarsätzen, einen Satz. Das Resultat jeder Wahrheitsoperation mit den Resultaten von Wahrheitsoperationen mit Elementarsätzen ist wieder das Resultat E i n e r Wahrheitsoperation mit Elementarsätzen.
Jeder Satz ist das Resultat von Wahrheitsoperationen mit Elementarsätzen.
…
""".strip()

TARGET_LANG = "English"


---
---

# Preprocessing

## Step 1: Canonical Translation Search

In [3]:
def canonical_info(text: str, k: int = 3) -> list[str]:
    prompt = f"""
    Find at least {k} published or widely cited translations of the text below.
    Return ONLY a JSON array (strings).

    TEXT:
    ---
    {SOURCE_TEXT}
    ---
    """
    resp = client.responses.create(
        model=MODEL_SEARCH,
        input=prompt,
        tools=[{"type": "web_search_preview"}],
    )
    return json.loads(resp.output_text)


## Step 2: Word-by-Word Definitions, Etymologies, Connotations

In [4]:
def word_metadata(text: str) -> dict[str, dict]:
    """
    Returns a dict that looks like:
        {
          "word1": {"definition": "...", "etymology": "...", "connotations": "..."},
          "word2": {...},
          ...
        }
    """

    prompt = f"""
    Provide a JSON object where each key is a distinct word
    from the text below, and each value is an object with
    fields "definition", "etymology", and "connotations".

    Return **ONLY** valid JSON — absolutely no extra commentary.

    TEXT:
    ---
    {SOURCE_TEXT}
    ---
    """

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )


word_info = word_metadata(SOURCE_TEXT)


## Step 3: Authorial Context

In [5]:
def author_context(text: str) -> dict:
    """
    Returns a dict like:
        {
          "author": "…",
          "life_span": "YYYY–YYYY",
          "era": "…",
          "notes": "…"
        }
    """

    prompt = f"""
    Give concise context about the author of the text below.
    Return **ONLY** a JSON object with the keys
      "author", "life_span", "era", "notes".
    If the exact year of the quoted translation is known, include it in 'notes'.

    TEXT:
    ---
    {SOURCE_TEXT}
    ---
    """

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )

author_info = author_context(SOURCE_TEXT)

## Step 4: Phonetics and Rhythm

In [6]:
def sound_profile(text: str) -> dict:
    """
    Returns a dict like:
        {
          "meter": "…",
          "notable_phonetics": "…",
          "observations": "…"
        }
    """

    prompt = f"""
    Analyse the phonetic and rhythmic qualities of the text below.
    Return **ONLY** a JSON object with keys
      "meter", "notable_phonetics", "observations".

    TEXT:
    ---
    {SOURCE_TEXT}
    ---
    """

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )


sound_info = sound_profile(SOURCE_TEXT)


## Combine Preprocessing Outputs

In [7]:
preproc_bundle = {
    "canonical_translations": canonical_info,
    "word_info":              word_info,
    "author_info":            author_info,
    "sound_info":             sound_info,
}

---
---
# Processing

## Step 1: Generate Translation from Preprocessing

In [9]:
def translate_once(text: str,
                   bundle: dict,
                   prior_translation: str | None = None) -> str:
    """
    Produce a translation informed by `bundle`, optionally improving a draft.
    """
    bundle_json = json.dumps(bundle,
                             ensure_ascii=False,
                             indent=2,
                             default=str)

    system_msg = {
        "role": "developer",
        "content": textwrap.dedent(f"""
            You are an academic translator, well-versed in translational theory.

            ## Context (JSON)
            ```json
            {bundle_json}
            ```

            • Preserve meaning, style, and sonic qualities where possible.
            • If a prior draft is provided, improve it; otherwise create a new one.
            • Output **only** the translation text—no commentary.
        """)
    }

    user_msg = {
        "role": "user",
        "content": (
            prior_translation
            if prior_translation
            else f"Translate into {TARGET_LANG}:\n\n{text}"
        )
    }

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=[system_msg, user_msg]
    )
    return resp.output_text.strip()


current_translation = translate_once(SOURCE_TEXT, preproc_bundle)

## Step 2: Judge First Translation

In [10]:
def judge_translation(src: str, draft: str) -> dict:
    """
    Returns {"positives": [...], "negatives": [...]}
    """

    prompt = f"""
    Evaluate the English translation against the source.
    Return **ONLY** a JSON object with two keys:
      "positives" : string list of strengths
      "negatives" : string list of weaknesses

    <source>
    {src}
    </source>

    <translation>
    {draft}
    </translation>
    """

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )

    try:
        return json.loads(resp.output_text)
    except json.JSONDecodeError as e:
        raise ValueError("Model did not return valid JSON; consider retrying.") from e


feedback = judge_translation(SOURCE_TEXT, current_translation)


## Step 3: Revise Translation from Feedback

In [11]:
def revise_translation(src: str,
                       draft: str,
                       critique: dict) -> str:
    """
    Improve `draft` using `critique`.
    Returns the revised translation as plain text.
    """

    messages = [
        {
            "role": "developer",
            "content": textwrap.dedent(f"""
                You are revising an English literary translation.

                ## Critique
                Positives → {critique['positives']}
                Negatives → {critique['negatives']}

                • Keep all positives intact.
                • Fix every negative issue.
                • Output **only** the revised translation text—no commentary,
                  no markdown.
            """)
        },
        {
            "role": "user",
            "content": draft
        }
    ]

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=messages
    )
    return resp.output_text.strip()


current_translation = revise_translation(
    SOURCE_TEXT,
    current_translation,
    feedback
)


## Step 4: *N* iterations

In [12]:
ITERATIONS   = 3       # set number of iterations

for i in range(1, ITERATIONS):
    fb   = judge_translation(SOURCE_TEXT, current_translation)
    prev = current_translation
    current_translation = revise_translation(SOURCE_TEXT, prev, fb)
    print(f"Iteration {i} done")

Iteration 1 done
Iteration 2 done


---
---

# Post-Processing

## Step 1: Translation + Canonical Critique

In [18]:
def compare_to_canon(final: str,
                     canon: list[str]) -> str:
    """
    Returns a short critique of `final` vs. canonical translations.
    """
    prompt = textwrap.dedent(f"""
        Compare the new translation to these canonical versions. Make these observations as concise and academically technical as possible. You are a research assistant to a brilliant translator, who needs to make edits quickly, not slog through your verbose comments. Be precise, identify strenghts and weaknesses. Only return the observations, no framing language of 'certainly,' or 'of course,' etc.

        <new_translation>
        {final}
        </new_translation>

        <canonical>
        {json.dumps(canon, ensure_ascii=False, indent=2, default=str)}
        </canonical>

        • Highlight unique strengths or weaknesses of the new version.
        • Mention where it surpasses or falls short of canon.
    """)

    resp = client.responses.create(
        model=MODEL_MAIN,
        input=prompt
    )
    return resp.output_text.strip()


analysis = compare_to_canon(current_translation, canonical_info)

print("### Final translation ###\n")
print(current_translation)
print("\n---\n")
print("### Comparative analysis ###\n")
print(analysis)


### Final translation ###

All propositions are results of truth-operations on elementary propositions.  
A truth-operation is the manner in which, out of elementary propositions, truth-functions arise.  
It follows from the essence of a truth-operation, just as a truth-function arises from elementary propositions, so too does a new truth-function arise from existing truth-functions. The result of each truth-operation on the results of truth-operations on elementary propositions is again the result of a truth-operation on elementary propositions. The result of any truth-operation performed on the results of truth-operations with elementary propositions is again the result of a single truth-operation on elementary propositions.  
Every proposition is the result of truth-operations on elementary propositions.  
…

---

### Comparative analysis ###

- The new translation is more explicit and repetitively schematic than canonical versions, sacrificing elegance for clarity.
- "Truth-operati

## Step 3: Final Edits

In [21]:
USER_EDITS = """
collapse repeated clauses
""".strip()

# e.g. “Make diction simpler in line 2”, “Keep rhyme scheme”, …

---
---

# Final Translation

In [22]:
def apply_edits(draft: str,
                canon_analysis: str = "",
                user_edits: str = "") -> str:
    """
    Improve `draft` by:
      • integrating observations from `canon_analysis`
      • applying user-supplied edits in `user_edits`
    Returns the revised translation (plain text).
    """
    if not (canon_analysis.strip() or user_edits.strip()):
        return draft

    dev_blocks = ["You are revising the translation."]

    if canon_analysis.strip():
        dev_blocks.append("## OBSERVATIONS FROM CANON COMPARISON\n"
                          + canon_analysis.strip())

    if user_edits.strip():
        dev_blocks.append("## USER EXTRA EDITS\n" + user_edits.strip())

    dev_blocks.append("Return **only** the final revised translation text.")

    messages = [
        {"role": "developer", "content": "\n\n".join(dev_blocks)},
        {"role": "user",      "content": draft}
    ]

    resp = client.responses.create(model=MODEL_MAIN, input=messages)
    return resp.output_text.strip()


final_translation = apply_edits(
    draft=current_translation,
    canon_analysis=analysis,
    user_edits=USER_EDITS
)

print("### Final-final translation ###\n")
print(final_translation)


### Final-final translation ###

All propositions are results of truth-operations on elementary propositions.  
A truth-operation is the manner in which truth-functions arise from elementary propositions.  
It follows from the nature of a truth-operation that, just as a truth-function arises from elementary propositions, so too can a new truth-function arise from existing truth-functions; the result of any truth-operation performed on results of truth-operations on elementary propositions is itself the result of a truth-operation on elementary propositions.  
…
